<a href="https://colab.research.google.com/github/abuwildanm/food-recognition/blob/master/Create_Custom_Dataset_From_Google_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Rename images & categories

In [18]:
import os
# use this to rename the categories of the folder generated by the chrome extension
# map_categories = {'mie_aceh - google penelusuran':'mie_aceh'}

# # Rename categories
# for before, after in map_categories.items():
#     os.rename('./{}'.format(before), './{}'.format(after))

In [ ]:
from glob import glob

print('Number of images')
# before processing the images, locate the dataset folder for each category to dataset_no_split folder, such as dataset_no_split/mie_aceh
categories = os.listdir('./dataset_no_split')
print(categories)
for cat in categories:
    print('{}: {}'.format(cat, len(glob('dataset_no_split/{}/*'.format(cat)))))

In [52]:
# Remove invalid images
all_image = glob('dataset_no_split/*/*')
for img_path in all_image:
    if os.path.getsize(img_path) == 0 and os.path.exists(img_path):
        os.remove(img_path)

In [ ]:
print('Number of valid images')
categories = os.listdir('./dataset_no_split')
for cat in categories:
    print('{}: {}'.format(cat, len(glob('dataset/{}/*'.format(cat)))))

In [68]:
# Rename each image from each category (ex: mie_aceh-1.jpg, mie_aceh-2.jpg, etc for category mie_aceh)
for cat in categories:
    for i, path in enumerate(sorted(glob('dataset_no_split/{}/*'.format(cat))), 1):
        dirname = os.path.dirname(path)
        src = path
        dst = os.path.join(dirname, '{}-{}.jpg'.format(cat, i))
        # Rename images
        os.rename(src, dst)

## Resize Images

In [ ]:
import tensorflow as tf

# The path of all image in dataset
image_path = glob('dataset_no_split/*/*') # change with desired path because some of the dataset already resized
# Resize process
image_size = (224, 224)
for path in image_path:
    # Load the image from path
    image = tf.keras.preprocessing.image.load_img(path)
    # Resize the image
    image = image.resize(image_size)
    # Save the resized image
    image.save(path)

## Create dataset

In [55]:
# Make train and test directory
os.makedirs('dataset/train', exist_ok=True)
os.makedirs('dataset/test', exist_ok=True)

# Make category directory in train and test
for cat in categories:
    os.makedirs('dataset/train/{}'.format(cat), exist_ok=True)
    os.makedirs('dataset/test/{}'.format(cat), exist_ok=True)

In [59]:
import numpy as np
import shutil

# Pick 20 percent of each category images as test set
for cat in categories:
    test_size = int(len(glob('dataset_no_split/{}/*'.format(cat))) * 0.2)
    all_cat_image = glob('dataset_no_split/{}/*'.format(cat))
    np.random.shuffle(all_cat_image)
    for img_path in sorted(all_cat_image):
        if len(os.listdir('dataset/test/{}'.format(cat))) <= test_size:
            shutil.move(img_path, 'dataset/test/{}'.format(cat))
        else:
            shutil.move(img_path, 'dataset/train/{}'.format(cat))

In [ ]:
categories_split = os.listdir('./dataset/train/')
print('Train Images')
for cat in categories_split:
    print('{}: {}'.format(cat, len(glob('dataset/train/{}/*'.format(cat)))))

print('='*20)
print('Test Images')
for cat in categories_split:
    print('{}: {}'.format(cat, len(glob('dataset/test/{}/*'.format(cat)))))

In [63]:
# Remove parent category directory
for cat in categories:
    shutil.rmtree('dataset_no_split/{}'.format(cat), ignore_errors=True)